# Gemini Powered Digital Asset Management

This notebook illustrates the beginnings of a basic digital asset management system that is powered by Google's Gemini API. The system is designed to support management of photography image libraries. In this notebook we explore importing images into a database, capturing and generating descriptive metadata, and searching the image library–all with the help of a Gemini chat agent.

## Image Library

At its core, the DAM consists of a collection of images that live on disk. A database is kept to store persistent data that is descriptive of the images in the library. Original assets are referenced by their file path on disk.

In [1]:
# Import database packages
import fastlite
from pydantic import BaseModel, RootModel, Field, ConfigDict
from pydantic.dataclasses import dataclass as pydantic_dataclass
from datetime import datetime

### Core Schema

The root table of our database keeps track of the assets in our system. It needs few fields:
* id: unique id for the image asset in the database
* image_path: unique file path to the original image on disk
* file_name: (optional) name of the image file
* file_type: (optional) file type of the orginal image on disk
  

In [2]:
@pydantic_dataclass
class Assets:
    # Configure the BaseModel to ignore any extra attributes given at creation
    model_config = ConfigDict(extra='ignore')
    id: int = Field(default=None)
    image_path: str = Field(default=None)
    file_name: str | None = Field(default=None)
    file_type: str | None = Field(default=None)

In [3]:
# Load/Create the database
db = fastlite.database('image_library.db')
# Create core table
assets = db.t.assets
if not assets.exists():
    assets = db.create(Assets)

### Metadata Schema

The metadata table in our database keeps record of select image metadata that is extracted from the original image file on disk. For simplicty, we're going to save our list of keywords as a comma seperated string.
* id: unique id for the image asset in the database
* capture_date: date the original image was created
* description: caption or description of the image
* keywords: a list of descriptive words or short phrases that describe the image
* creator: name of the image creator
* person_in_image: a list of names labeling people shown in the image
* location: location shown or where image was originally taken
* city: city shown or where image was originally taken
* state: state shown or where image was originally taken

In [4]:
# Define metadata schema
@pydantic_dataclass
class AssetMetadata:
    # Configure the BaseModel to ignore any extra attributes given at creation
    model_config = ConfigDict(extra='ignore')
    id: int = Field(default=None)
    capture_date: datetime | None = Field(default=None)
    description: str | None = Field(default=None)
    keywords: str | None = Field(default=None)
    creator: str | None = Field(default=None)
    person_in_image: str | None = Field(default=None)                  
    location: str | None = Field(default=None)
    city: str | None = Field(default=None)
    state: str | None = Field(default=None)

In [5]:
# Create metadata table
asset_metadata = db.t.asset_metadata
if not asset_metadata.exists():
    asset_metadata = db.create(AssetMetadata)

### Generative Schema

The generative table in our database keeps record of generated content created by the Gemini model. For simplicty, we're going to save our list of keywords as a comma seperated string.
* id: unique id for the image asset in the database
* description: generated description of the image
* keywords: a list of generated keywords that describe the image
* style: style of the image as genai sees it (from taxonomy)
* mood: mood of the image as genai sees it (from taxonomy)

In [6]:
# Define generative schema
@pydantic_dataclass
class GenerativeMetadata:
    # Configure the BaseModel to ignore any extra attributes given at creation
    model_config = ConfigDict(extra='ignore')
    id: int = Field(default=None)
    description: str = Field(default=None)
    keywords: str = Field(default=None)
    style: str = Field(default=None)
    mood: str = Field(default=None)

In [7]:
# Create generative table
generative_metadata = db.t.generative_metadata
if not generative_metadata.exists():
    generative_metadata = db.create(GenerativeMetadata)

### Embedding Schema

The embedding table in our database keeps record of embeddings generated by the Gemini model.
* id: unique id for the image asset in the database
* genai_description_vector: embedding representing the generated description of the image

In [8]:
# Define embedding schema
@pydantic_dataclass
class Embeddings:
    # Configure the BaseModel to ignore any extra attributes given at creation
    model_config = ConfigDict(extra='ignore')
    id: int = Field(default=None)
    genai_description_vector: bytes = Field(default=None)

In [9]:
# Create embedding table
embeddings = db.t.embeddings
if not embeddings.exists():
    embeddings = db.create(Embeddings)

In [10]:
# Our database is now set up, let's view its schema
print(db.schema)

CREATE TABLE [assets] (
   [id] INTEGER PRIMARY KEY,
   [image_path] TEXT,
   [file_name] TEXT,
   [file_type] TEXT
);
CREATE TABLE [asset_metadata] (
   [id] INTEGER PRIMARY KEY,
   [capture_date] TEXT,
   [description] TEXT,
   [keywords] TEXT,
   [creator] TEXT,
   [person_in_image] TEXT,
   [location] TEXT,
   [city] TEXT,
   [state] TEXT
);
CREATE TABLE [generative_metadata] (
   [id] INTEGER PRIMARY KEY,
   [description] TEXT,
   [keywords] TEXT,
   [style] TEXT,
   [mood] TEXT
);
CREATE TABLE [embeddings] (
   [id] INTEGER PRIMARY KEY,
   [genai_description_vector] BLOB
);
CREATE TABLE sqlite_stat1(tbl,idx,stat);
CREATE TABLE sqlite_stat4(tbl,idx,neq,nlt,ndlt,sample);


### Gemini API Setup

In [11]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
from PIL import Image
import json
import numpy as np
import time

In [12]:
# Get our api key
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
# Set up gemini client
client = genai.Client(api_key=GOOGLE_API_KEY)

#### Gemini Image Descripton Content Generation

We'll use Gemini as our vision model to generate new descriptions of our images. We'll pass our image to the model and ask for a description, list of keywords, style, and mood. Let's define our json respsonse schema.

In [13]:
# Define the model response schema
class DescriptionResponseSchema(BaseModel):
    description: str  # Long description of the image
    keywords: list[str]  # List of keywords describing the image
    style: str  # The style of the image
    mood: str  # The mood of the image

Two of our fields, style and mood, will use a taxonomy. Two taxonomies were generated in the Taxonomy.ipynb and saved to json files on disk. Let's load our taxonomies so we can use them with the model.

In [14]:
# Load taxonomy lists
with open('../data/style_taxonomy.json') as style_f:
    STYLE_TAX = json.load(style_f)
with open('../data/mood_taxonomy.json') as mood_f:
    MOOD_TAX = json.load(mood_f)

In [15]:
print(STYLE_TAX['taxonomy'], MOOD_TAX['taxonomy'])

['Abstract', 'Airy', 'Artistic', 'Authentic', 'Balanced', 'Bold', 'Bright', 'Candid', 'Chiaroscuro', 'Cinematic', 'Classic', 'Clean', 'Colorful', 'Contemporary', 'Contrasty', 'Cozy', 'Dark', 'Delicate', 'Detailed', 'Dramatic', 'Dreamy', 'Dynamic', 'Earthy', 'Elegant', 'Emotional', 'Energetic', 'Fine-Art', 'Flat', 'Flowing', 'Folk', 'Fragmented', 'Geometric', 'Glamorous', 'Gritty', 'High-Key', 'Illustrative', 'Intimate', 'Layered', 'Light', 'Lomo', 'Low-Key', 'Minimalist', 'Monochromatic', 'Moody', 'Natural', 'Nostalgic', 'Painterly', 'Playful', 'Pop', 'Rustic', 'Soft', 'Surreal', 'Vintage'] ['Melancholy', 'Joyful', 'Serene', 'Agitated', 'Tranquil', 'Frightened', 'Peaceful', 'Enraged', 'Content', 'Disgusted', 'Optimistic', 'Pessimistic', 'Romantic', 'Lonely', 'Playful', 'Solemn', 'Mysterious', 'Bored', 'Excited', 'Guilty', 'Empowering', 'Shameful', 'Reflective', 'Jealous', 'Dreamy', 'Betrayed', 'Vibrant', 'Nostalgic', 'Tense', 'Comforting', 'Suspicious', 'Hopeful', 'Despairing', 'Curiou

Our model needs instructions, we'll list some guidelines and include the taxonomies for style and mood.

In [16]:
SYSTEM_INSTRUCTIONS_FOR_DESCRIPTIONS = f"""
        Write a caption for the images you see using The Associated Press standards for photo captions.
        Be very descriptive and thoroughly describe the image using 100-500 words.
        The description you write will be used for the search and retrieval of visual assets in a digital asset management system.
        I may provide you with additional context, but not always.
        Additional context might be a description of the image, date, location, keywords, and persons shown in the image.
        Use all additional context that I provide you about the image to inform the description you write.
        You must include the date when it is provided to you, otherwise do not guess the date.
        Include the location when it is provided to you.
        Generate a list of keywords or short descriptive phrases that relate to the image.
        Keywords can be one to two words long and may describe the contents, feeling, style, or mood of the image.
        Generate no more than 20 keywords.
        Label the style and mood of the image using only keywords from the taxonomy.
        Style taxonomy: {STYLE_TAX['taxonomy']}
        Mood taxonomy: {MOOD_TAX['taxonomy']}
        """

In [17]:
# Configure Gemini model for description generation
GEMINI_DESCRIPTION_CONFIG = types.GenerateContentConfig(safety_settings=None,
                                                        system_instruction=SYSTEM_INSTRUCTIONS_FOR_DESCRIPTIONS,
                                                        max_output_tokens=2048,
                                                        temperature=0.3,
                                                        top_p=0.6,
                                                        top_k=32,
                                                        presence_penalty=0.3,
                                                        frequency_penalty=0.3,
                                                        response_mime_type='application/json',
                                                        response_schema=DescriptionResponseSchema
                                                       )

We're going to set up a one-shot Gemini model for the description generation. We'll give it the image as well as the image's metadata to give it some context.

In [18]:
# Set up a Gemini model for inference
class GeminiDescription:
    def __init__(self, _image: Image.Image, _image_metadata: AssetMetadata, _record: Assets):
        self.image = _image
        self.image_metadata = _image_metadata
        self.record = _record
        self.description = self.generate_description()

    def generate_description(self) -> GenerativeMetadata:    
        _result = client.models.generate_content(model='gemini-2.0-flash-exp',
                                                 config=GEMINI_DESCRIPTION_CONFIG,
                                                 contents=[self.image_metadata, self.image])
        _result_dict = json.loads(_result.text)
        
        return GenerativeMetadata(id=self.record.id,
                                  description=_result_dict['description'],
                                  keywords=", ".join(_result_dict['keywords']),
                                  style=_result_dict['style'],
                                  mood=_result_dict['mood'])

Let's set up an additional Gemini model to generate an embedding of the generated description.

In [19]:
# Configure Gemini model for embedding
class GeminiEmbedding:
    def __init__(self, _description: str, _record: Assets):
        self.description = _description
        self.record = _record
        self.embedding = self.generate_embedding()

    def generate_embedding(self) -> Embeddings:
        result = client.models.embed_content(model="models/text-embedding-004",
                                             contents=self.description)
        _embedding = np.array(result.embeddings[0].values).astype(np.float32).tobytes()

        return Embeddings(id=self.record.id,
                          genai_description_vector=_embedding)

## Import

Importing an image into the library is the first step in our interaction with the DAM. The import workflow has a number of steps that are facilitated by our Gemini agent. 

* First, we start a chat with our agent by providing it an image. In this workflow, we will be selecting images on disk, apposed to uploading to a platform.

* Once we kick-off the workflow, our agent with create a new record in the database for the image we are currently importing. It assigns our image an unique asset id and extracts some key information that may be embedded in the file.

* From there, our agent will generate new descriptive content for the image by passing it to Gemini model for vision analysis.

* Finally, our agent sends the newly generated description to the Gemini embedding model to be vectorized. These embeddings will allow for semantic search of our image library.

Let's start by defining some tools to use in our workflow.

In [20]:
# Import packages for import workflow
from pillow_metadata.metadata import Metadata

In [21]:
# Open the image from the file path
def open_image(_image_path: str) -> Image.Image:
    _image = Image.open(_image_path)
    # resize the image
    return _image

# Read some metadata from the image file
def read_metadata(_image: Image.Image, _record: Assets) -> AssetMetadata:
    _meta = Metadata(_image)
    _extracted = {'capture_date': _meta.metadata.xmp.CreateDate,
                  'description': _meta.metadata.dc.description,
                  'keywords': ", ".join(_keywords) if (_keywords := _meta.metadata.dc.subject) else _keywords,
                  'creator': _meta.metadata.exif.Artist,
                  'person_in_image': ", ".join(_person_in_image) if (_person_in_image := _meta.metadata.Iptc4xmpExt.PersonInImage) else _person_in_image,
                  'location': _meta.metadata.Iptc4xmpCore.Location,
                  'city': _meta.metadata.photoshop.City,
                  'state': _meta.metadata.photoshop.State}
    
    return AssetMetadata(**_extracted, id=_record.id)

In [22]:
# Functions to help us insert and update records in the database
def insert_asset(_asset: Assets, _update: bool) -> Assets | None:
        
    # check if this asset is already in the database
    if not (query := db.q(f"SELECT id FROM assets WHERE file_name == '{_asset.file_name}'")):
        _record = assets.insert(_asset)
        
        return Assets(**_record)
        
    elif _update:
        _asset.id = query[0]['id']
        _record = assets.update(_asset)
        
        return Assets(**_record)

    return None

def insert_metadata(_asset_metadata: AssetMetadata) -> AssetMetadata:
    # if asset metadata is not in the database for this record, then insert it
    if not db.q(f"SELECT id FROM asset_metadata WHERE id == {_asset_metadata.id}"):
        _record = asset_metadata.insert(_asset_metadata)
    # otherwise update the record in the database
    else:
        _record = asset_metadata.update(_asset_metadata)

    return _record

def insert_genai_description(_genai_desc: GenerativeMetadata) -> GenerativeMetadata:
    # if genai description is not in the database for this record, then insert it
    if not db.q(f"SELECT id FROM generative_metadata WHERE id == {_genai_desc.id}"):
        _record = generative_metadata.insert(_genai_desc)
    # otherwise update the record in the database
    else:
        _record = generative_metadata.update(_genai_desc)
    
    return _record

def insert_embedding(_embedding: Embeddings) -> Embeddings:
    # if embedding is not in the database for this record, then insert it
    if not db.q(f"SELECT id FROM embeddings WHERE id == {_embedding.id}"):
        _record = embeddings.insert(_embedding)
    # otherwise update the record in the database
    else:
        _record = embeddings.update(_embedding)
    
    return _record

Let's go through each step of the import.

In [23]:
# Wrap our tools in a import workflow
def import_image(_image_path: str, _update=False):
    # Create an Assets object for our image
    _asset = Assets(image_path=_image_path, file_name=_image_path.split('/')[-1])
    # let's open the image
    _image = open_image(_asset.image_path)
    # check if the image already exists in the database
    if _record := insert_asset(_asset, _update):
        # get the image's embedded metadata
        _img_meta = read_metadata(_image, _record)
        # insert the metadata into the database
        _meta_record = insert_metadata(_img_meta)
        # generate image descriptions
        time.sleep(1)
        _genai_desc = GeminiDescription(_image,
                                        RootModel[AssetMetadata](_meta_record).model_dump_json(exclude_none=True),
                                        _record
                                       ).description
        # insert the generated image descriptions into the database
        _genai_desc_record = insert_genai_description(_genai_desc)
        # generate an embedding
        _embedding = GeminiEmbedding(_description=_genai_desc.description, _record=_record).embedding
        # insert embedding data into the database
        _embedding_record = insert_embedding(_embedding)

        return f"Success! Asset with file name {_asset.file_name} has been updated in the database."
        
    elif not _update:
        return (f"Asset with file name {_asset.file_name} already exists in database. "
                "If you would like to update the record for this asset, submit your request again with the update flag set to True.")


### Test out import workflow with a single image

In [24]:
test_image_path = '/Users/peterjakubowski/Desktop/Jupyter/google_genai/datasets/image_library/20211224_P8J8463.jpg'

result = import_image(test_image_path, _update=True)

print(result)

Success! Asset with file name 20211224_P8J8463.jpg has been updated in the database.


## Search

Searching our image library is conversational. Our agent is going to help us find images in our library based on our query.

Our agent has a host of tools at its disposal to process our query. Whatever our query, our agent will try its best to understand what we're looking for.

Let's begin with a simple sql query tool.

In [25]:
# Sql search
def search_image_library_sql(_sql_query: str) -> str:
    """
    Search the image library database using sql queries.

    Example: 'SELECT * FROM assets WHERE id == 1'

    Args:
        _sql_query (string): A sql command to execute on the image library database.
    
    """

    _query_results = db.q(_sql_query)

    return _query_results

In [26]:
search_image_library_sql("SELECT * FROM assets WHERE id == 7")

[{'id': 7,
  'image_path': '/Users/peterjakubowski/Desktop/Jupyter/google_genai/datasets/image_library/20211224_P8J8463.jpg',
  'file_name': '20211224_P8J8463.jpg',
  'file_type': None}]

### Topic Search

Does our query include any keywords or phrases that exist in our taxonomy? Our agent will decide if it can extract any search terms that can be used to query our database. Generated summaries are used to better understand what we're asking.

### Vector Search

Our query is vectorized and used to search similarities in our database. The agent might do a few things to better produce an embedding of our query that matches embeddings in our database. It might ask itself if our query is a description of an image and whether or not it can be modified to better describe images in our database. Our agent may summarize our query and even expand on it. Perhaps our agent goes a step further and transforms our query into a prompt that could generate images of our query, then that prompt in vectorized and used in a similarity search. For now, we will define a simple nearest neighbors search based on the generated descriptions in our database.

In [27]:
from sklearn.neighbors import NearestNeighbors

In [28]:
class Neighbors:

    def __init__(self):
        self.model = None
        self.index_map = {}
        self.train_model()

    def train_model(self):
        vectors_query = db.q("SELECT * FROM embeddings")
        self.index_map = {}
        vectors = []
        for i, v in enumerate(vectors_query):
            self.index_map[i] = v['id']
            vectors.append(np.frombuffer(v['genai_description_vector'], dtype=np.float32))

        neighbors = NearestNeighbors(n_neighbors=5, radius=1.0)

        self.model = neighbors.fit(np.array(vectors))

    def search(self, search_vector):
        search_result = self.model.radius_neighbors(search_vector.reshape(1, -1), sort_results=True)
        return search_result

In [29]:
neighbors = Neighbors()
neighbors.model

NearestNeighbors()

In [30]:
def search_image_library_semantic(_query_text: str) -> list[Assets]:
    """
    Search the image library using vector search.
    The query text should describe an image that we're looking for.
    Accepts query as text, generates an embedding representation of the query,
    and returns results from a nearest neighbors search.

    Args:
        _query (string): Query text string
        
    """
    
    print(f"semantic query: {_query_text}")
    
    # Generate an embedding of our query text
    _response = client.models.embed_content(model="models/text-embedding-004",
                                            contents=_query_text)
    # Format our embedding as a numpy array
    _embedding = np.array(_response.embeddings[0].values).astype(np.float32)
    # Query our nearest neighbors model
    _results = neighbors.search(search_vector=_embedding)[1].tolist()[0]
    
    return [Assets(**search_image_library_sql(f"SELECT * FROM assets WHERE id == {neighbors.index_map[_res]}")[0]) for _res in _results]


In [31]:
for res in search_image_library_semantic("Cactus plants"):
    print(res)


semantic query: Cactus plants
Assets(id=3, image_path='/Users/peterjakubowski/Desktop/Jupyter/google_genai/datasets/image_library/20190224_Sandcastle_Cactus_0036.jpg', file_name='20190224_Sandcastle_Cactus_0036.jpg', file_type=None)
Assets(id=6, image_path='/Users/peterjakubowski/Desktop/Jupyter/google_genai/datasets/image_library/2018-12-22_Peruvian_Torch_0468.jpg', file_name='2018-12-22_Peruvian_Torch_0468.jpg', file_type=None)


### Set up Function Calling Tools

Declare functions that our chat model will have access to.

In [32]:
import_image_func = types.FunctionDeclaration(
    name='import_image',
    description='Open an image, get information and metadata about the image, and import it into a database.',
    parameters=types.Schema(
        type=types.Type('OBJECT'),
        properties={
            "_image_path": types.Schema(type=types.Type('STRING')),
            "_update": types.Schema(type=types.Type('BOOLEAN'))
        })
)

search_image_library_sql_func = types.FunctionDeclaration(
    name='search_image_library_sql',
    description=("Search the image library sqlite database using sql queries. "
                 f"Database schema: {db.schema}"
                ),
    parameters=types.Schema(
        type=types.Type('OBJECT'),
        properties={
            "_sql_query": types.Schema(type=types.Type('STRING'))
        })
)

search_image_library_semantic_func = types.FunctionDeclaration(
    name='search_image_library_semantic',
    description=("Search the image library using vector search."
                 "The query text should describe an image that we're looking for. "
                 "Accepts query as text, generates an embedding representation of the query, "
                 "and returns results from a nearest neighbors search."
                ),
    parameters=types.Schema(
        type=types.Type('OBJECT'),
        properties={
            "_query_text": types.Schema(type=types.Type('STRING'))
        })
)

tools = types.Tool(function_declarations=[import_image_func,
                                          search_image_library_sql_func,
                                          search_image_library_semantic_func
                                         ]
                  )

### Start a chat

We're ready to instruct our Gemini agent.

In [33]:
CHAT_SYSTEM_INSTSTRUCTION = ("You are my image library database administrator. "
                             "I provide a path to an image and you import it into the database. "
                             "I ask a question about the image library, you execute sql statements to query the database. "
                             "I ask for an image report, you generate a report with an image's information. "
                            )

In [34]:
# Configure our model
CHAT_MODEL_CONFIG = types.GenerateContentConfig(safety_settings=None,
                                           tools=[tools],
                                           tool_config=types.ToolConfig(
                                               function_calling_config=types.FunctionCallingConfig(
                                                   mode=types.FunctionCallingConfigMode("AUTO"))),
                                           system_instruction=CHAT_SYSTEM_INSTSTRUCTION,
                                           max_output_tokens=2048,
                                           temperature=0.3,
                                           top_p=0.6,
                                           top_k=32,
                                           presence_penalty=0.3,
                                           frequency_penalty=0.3,
                                           automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True,
                                                                                                           maximum_remote_calls=None
                                                                                                           )
                                           )

In [42]:
# Start a new chat
chat = client.chats.create(model='gemini-2.0-flash-exp',
                           config=CHAT_MODEL_CONFIG)

In [43]:
# Build our two-part prompt to begin our chat with
prompt_text = types.Part.from_text(text="Hey Gemini! I have an image to import into my database. Do not update the record if it already exists. Please help!")
prompt_image = types.Part.from_text(text=test_image_path)

In [46]:
# Send our message and get a response
response = chat.send_message(message=[prompt_text, prompt_image])

In [47]:
# Let's look at our first response
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'_update': False, '_image_path': '/Users/peterjakubowski/Desktop/Jupyter/google_genai/datasets/image_library/20211224_P8J8463.jpg'}, name='import_image'), function_response=None, inline_data=None, text=None)], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.001403958189721201, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'>, probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'>, probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category=<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 'HARM_CATEGORY_DANGEROUS_CONTENT'>

In [48]:
def process_response(_response):
    
    if function_calls := _response.function_calls:
        results = []
        content = ""
        for call in function_calls:
            if call.name == 'import_image':
                content = import_image(_image_path=call.args['_image_path'], _update=call.args['_update'])
            elif call.name == 'search_image_library_sql':
                content = search_image_library_sql(_sql_query=call.args['_sql_query'])
            elif call.name == 'search_image_library_semantic':
                content = search_image_library_semantic(_query_text=call.args['_query_text'])
                
            results.append(
                types.Part.from_function_response(
                    name=call.name,
                    response={'content': content}
                )
            )

        time.sleep(1)
        return process_response(chat.send_message(results))

    return _response.text

    

In [49]:
# Let's chat for a while
do_some_work = True

while do_some_work:
            
    print(f"[agent]: {process_response(response)}")

    user_input = input("[user]: ")

    if user_input == 'STOP':
        do_some_work = False

    else:
        response = chat.send_message(message=user_input)

[agent]: OK. It looks like that image already exists in the database. I did not update the record.



[user]:  Thanks! Please update the record for me :-)


[agent]: OK. I've updated the record for you.



[user]:  Generate an image report with the image's information.


[agent]: OK. Here's the image report:

**File Path:** /Users/peterjakubowski/Desktop/Jupyter/google_genai/datasets/image_library/20211224_P8J8463.jpg
**File Name:** 20211224_P8J8463.jpg
**File Type:** None
**Capture Date:** 2021-12-24T20:26:21.350000
**Description:** A studio shot of a Yule Log cake on a white cake stand, photographed on December 24, 2021. The cake is a chocolate swiss roll with a creamy white filling, covered in chocolate frosting that has been textured to resemble tree bark. The cake is garnished with fresh rosemary sprigs, bright red cranberries, and sliced almonds. The cake is sitting on a bed of crushed chocolate cookies to resemble dirt. The background is a white tiled wall and a white surface.
**Keywords:** Yule Log, cake, dessert, chocolate, frosting, rosemary, cranberries, almonds, cookies, Christmas, holiday, sweet, baked, treat, festive
**Creator:** Peter Jakubowski
**Person in Image:** None
**Location:** None
**City:** None
**State:** None
**Style:** Bright

[user]:  Awesome!


[agent]: Is there anything else I can help you with?



[user]:  STOP
